In [1]:
# Import statements

import pandas as pd
import matplotlib.pyplot as plt

# Inspection and Strategy

## 1: Load and Inspect datasets

In [8]:
import os
print(os.getcwd())

/Users/mariarumpf/thesis-electricity-demand-forecasting/notebooks


In [14]:
electricity_df = pd.read_csv("../data/raw/electricity_data_germany.csv", sep = ";", low_memory = False)
weather_df = pd.read_csv("../data/raw/weather_data_germany.csv", sep = ";", low_memory = False)

# low_memory = False in pd.read_csv controls how Pandas handles data types when reading a large file
# Pandas defaultly infers column data types in chunks to save memory, causing mixed data types
# this forces Pandas to read the entire file first before deciding on data types

In [10]:
print(electricity_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175167 entries, 0 to 175166
Data columns (total 16 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   utc_timestamp                         175167 non-null  object 
 1   cet_cest_timestamp                    175167 non-null  object 
 2   DE_load_actual_entsoe_transparency    175167 non-null  object 
 3   DE_load_forecast_entsoe_transparency  175071 non-null  object 
 4   DE_solar_capacity                     175167 non-null  int64  
 5   DE_solar_generation_actual            174783 non-null  object 
 6   DE_solar_profile                      174783 non-null  float64
 7   DE_wind_capacity                      175167 non-null  int64  
 8   DE_wind_generation_actual             174869 non-null  object 
 9   DE_wind_profile                       174869 non-null  float64
 10  DE_wind_offshore_capacity             175167 non-null  int64  
 11  

Observations:
Missing values in DE_load_forecast_entsoe_transparency, DE_solar_generation_actual, and DE_wind_generation_actual

Can ignore CET timestamp, stick to UCT (similar to weather data which needs to be merged)
Forecast transparency is a row that needs to be removed - cannot be a feature! But can we kept for later (when comparing our predictions)

Starts in 2014, so make sure the overlap works

In [11]:
missing_values = electricity_df.isnull().sum()
print(missing_values[missing_values > 0])

DE_load_forecast_entsoe_transparency     96
DE_solar_generation_actual              384
DE_solar_profile                        384
DE_wind_generation_actual               298
DE_wind_profile                         298
DE_wind_offshore_generation_actual      298
DE_wind_offshore_profile                298
DE_wind_onshore_generation_actual       288
DE_wind_onshore_profile                 288
dtype: int64


In [12]:
missing_rows = electricity_df[electricity_df.isnull().any(axis = 1)]
print(missing_rows.index.tolist())

[5539, 5540, 5541, 5542, 5543, 5544, 5545, 5546, 5547, 5548, 5549, 5550, 5551, 5552, 5553, 5554, 5555, 5556, 5557, 5558, 5559, 5560, 5561, 5562, 5563, 5564, 5565, 5566, 5567, 5568, 5569, 5570, 5571, 5572, 5573, 5574, 5575, 5576, 5577, 5578, 5579, 5580, 5581, 5582, 5583, 5584, 5585, 5586, 5587, 5588, 5589, 5590, 5591, 5592, 5593, 5594, 5595, 5596, 5597, 5598, 5599, 5600, 5601, 5602, 5603, 5604, 5605, 5606, 5607, 5608, 5609, 5610, 5611, 5612, 5613, 5614, 5615, 5616, 5617, 5618, 5619, 5620, 5621, 5622, 5623, 5624, 5625, 5626, 5627, 5628, 5629, 5630, 5631, 5632, 5633, 5634, 49591, 49592, 49593, 49594, 49595, 49596, 49597, 49598, 49599, 49600, 49601, 49602, 49603, 49604, 49605, 49606, 49607, 49608, 49609, 49610, 49611, 49612, 49613, 49614, 49615, 49616, 49617, 49618, 49619, 49620, 49621, 49622, 49623, 49624, 49625, 49626, 49627, 49628, 49629, 49630, 49631, 49632, 49633, 49634, 49635, 49636, 49637, 49638, 49639, 49640, 49641, 49642, 49643, 49644, 49645, 49646, 49647, 49648, 49649, 49650, 496

No solar generation (actual and profile) reported for the 28th of February, 2015
No solar (2x), wind (2x on shore and off shore) reported on 2016-06-01 and 2016-10-27 and 2016-12-09
No wind reported on 2016-11-03 (couple of hours)

How is the target variable behaving here? how can we impute (careful of cycles!)

No forecasted load reported on 2018-09-24 (not so important)

Target variable is available throughout consistently (yay!)

In [15]:
print(weather_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52584 entries, 0 to 52583
Data columns (total 4 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   utc_timestamp                    52584 non-null  object 
 1   DE_temperature                   52584 non-null  float64
 2   DE_radiation_direct_horizontal   52584 non-null  object 
 3   DE_radiation_diffuse_horizontal  52584 non-null  object 
dtypes: float64(1), object(3)
memory usage: 1.6+ MB
None


Appears complete with no missing values, but there are 0s between 16h and 6h every 24h cycle in the dataset for the two radiation features. How can I still use it, and get it to recognize them as non-null values? Or should I assume there is no radiation during those times (unlikely)

In [16]:
print(electricity_df.head())

          utc_timestamp        cet_cest_timestamp  \
0  2015-01-01T07:15:00Z  2015-01-01T08:15:00+0100   
1  2015-01-01T07:30:00Z  2015-01-01T08:30:00+0100   
2  2015-01-01T07:45:00Z  2015-01-01T08:45:00+0100   
3  2015-01-01T08:00:00Z  2015-01-01T09:00:00+0100   
4  2015-01-01T08:15:00Z  2015-01-01T09:15:00+0100   

  DE_load_actual_entsoe_transparency DE_load_forecast_entsoe_transparency  \
0                            40998.2                             42497.11   
1                            41120.9                             42458.48   
2                           41476.39                             43085.68   
3                            42120.4                             44127.67   
4                           42624.45                              44906.5   

   DE_solar_capacity DE_solar_generation_actual  DE_solar_profile  \
0              37248                      14.18            0.0004   
1              37248                      49.02            0.0013   
2          

In [17]:
print(weather_df.head())

          utc_timestamp  DE_temperature DE_radiation_direct_horizontal  \
0  2014-01-01T00:00:00Z          -0.568                              0   
1  2014-01-01T01:00:00Z          -0.509                              0   
2  2014-01-01T02:00:00Z          -0.436                              0   
3  2014-01-01T03:00:00Z          -0.257                              0   
4  2014-01-01T04:00:00Z          -0.082                              0   

  DE_radiation_diffuse_horizontal  
0                               0  
1                               0  
2                               0  
3                               0  
4                               0  


## 2: Check Timestamp Formats

In [18]:
print("Electricity Time Range")
print(electricity_df['utc_timestamp'].sort_values().head())
print(electricity_df['utc_timestamp'].sort_values().tail())

print("Weather Time Range")
print(weather_df["utc_timestamp"].sort_values().head())
print(weather_df["utc_timestamp"].sort_values().tail())

# sorting just in case, so we can see the earliest and latest entries

Electricity Time Range
0    2015-01-01T07:15:00Z
1    2015-01-01T07:30:00Z
2    2015-01-01T07:45:00Z
3    2015-01-01T08:00:00Z
4    2015-01-01T08:15:00Z
Name: utc_timestamp, dtype: object
175162    2019-12-30T21:45:00Z
175163    2019-12-30T22:00:00Z
175164    2019-12-30T22:15:00Z
175165    2019-12-30T22:30:00Z
175166    2019-12-30T22:45:00Z
Name: utc_timestamp, dtype: object
Weather Time Range
0    2014-01-01T00:00:00Z
1    2014-01-01T01:00:00Z
2    2014-01-01T02:00:00Z
3    2014-01-01T03:00:00Z
4    2014-01-01T04:00:00Z
Name: utc_timestamp, dtype: object
52579    2019-12-31T19:00:00Z
52580    2019-12-31T20:00:00Z
52581    2019-12-31T21:00:00Z
52582    2019-12-31T22:00:00Z
52583    2019-12-31T23:00:00Z
Name: utc_timestamp, dtype: object


Observations:

Datasets use ISO 8601 timestamps

Coverage: electricity Jan 1 2015 to Dec 30, 2019; weather Jan 1 2014 to Dec 31, 2019 (WEATHER DATA FULLY OVERLAPS!)

In [19]:
# Conversion to Datetime Format
# This will help later, because right now they are just stored as strings. With datetime format, some additional operations can be done for merging + upsampling

electricity_df['utc_timestamp'] = pd.to_datetime(electricity_df['utc_timestamp'], errors = "coerce") # coercing errors safely converts invalid formats to NaT (not a time) for future catching
weather_df['utc_timestamp'] = pd.to_datetime(weather_df['utc_timestamp'], errors = "coerce")

print("Checking types")
print(electricity_df['utc_timestamp'].dtype)
print(weather_df['utc_timestamp'].dtype)
print()

print("Checking Missing Timestamps")
print(electricity_df['utc_timestamp'].isna().sum())
print(weather_df['utc_timestamp'].isna().sum())

Checking types
datetime64[ns, UTC]
datetime64[ns, UTC]

Checking Missing Timestamps
0
0


Proper format and no failed conversion! Can now to resampling, merging and feature engineering

## 3: Missing Values

In [22]:
weather_df.head()


,utc_timestamp,DE_temperature,DE_radiation_direct_horizontal,DE_radiation_diffuse_horizontal,hour
0,2014-01-01 00:00:00+00:00,-0.568,0,0,0
1,2014-01-01 01:00:00+00:00,-0.509,0,0,1
2,2014-01-01 02:00:00+00:00,-0.436,0,0,2
3,2014-01-01 03:00:00+00:00,-0.257,0,0,3
4,2014-01-01 04:00:00+00:00,-0.082,0,0,4


In [24]:
print(weather_df.dtypes)
print(weather_df["DE_radiation_direct_horizontal"].unique()[:10])

utc_timestamp                      datetime64[ns, UTC]
DE_temperature                                 float64
DE_radiation_direct_horizontal                  object
DE_radiation_diffuse_horizontal                 object
hour                                             int32
dtype: object
['0' '0.3239' '52.358' '192.939' '450.554' '663.742' '612.468' '331.142'
 '81.016' '0.2006']


In [26]:
# Investigating Radiation Zeros

# Count and percentage of zeros in radiation columns

weather_df["DE_radiation_direct_horizontal"] = pd.to_numeric(weather_df["DE_radiation_direct_horizontal"], errors="coerce")
weather_df["DE_radiation_diffuse_horizontal"] = pd.to_numeric(weather_df["DE_radiation_diffuse_horizontal"], errors="coerce")

for col in ["DE_radiation_direct_horizontal", "DE_radiation_diffuse_horizontal"]:
    zero_count = (weather_df[col] == 0).sum()
    total_count = len(weather_df)
    percent = zero_count / total_count * 100
    print(f"{col}: {zero_count} zeros ({percent:.2f}%)")

print()
# When zeros occur:
weather_df['hour'] = weather_df['utc_timestamp'].dt.hour

for col in ['DE_radiation_direct_horizontal', 'DE_radiation_diffuse_horizontal']:
    print(f"\nZero Distribution by hour for {col}:")
    zero_hours = weather_df[weather_df[col] == 0]['hour'].value_counts().sort_index()
    print(zero_hours)

DE_radiation_direct_horizontal: 22850 zeros (43.45%)
DE_radiation_diffuse_horizontal: 22497 zeros (42.78%)


Zero Distribution by hour for DE_radiation_direct_horizontal:
hour
0     2191
1     2191
2     1920
3     1471
4     1108
5      742
6      253
16     459
17     874
18    1243
19    1634
20    2191
21    2191
22    2191
23    2191
Name: count, dtype: int64

Zero Distribution by hour for DE_radiation_diffuse_horizontal:
hour
0     2191
1     2191
2     1851
3     1442
4     1085
5      709
6      170
16     413
17     850
18    1225
19    1606
20    2191
21    2191
22    2191
23    2191
Name: count, dtype: int64


Basically this means:

About 43% of rows are zero in both radiation columns, which is about 12 hours of darkness per day (on average)

Radiation is consistently 0 from 20:00 to 6:00, and the transition periods have a tapering pattern (sunrise/sunset). Midday hours (7:00 to 17:00) almost never contain zeros

Therefore:

Don't treat 0s as missing

Add a binary is_daylight column (therefore when radiation is 0, is because it's night, not cloudy, and can help learn conditional patterns )

In [27]:
# Adding a binary column

weather_df['is_daylight'] = weather_df['hour'].between(6, 19).astype(int)

weather_df[100:105]


,utc_timestamp,DE_temperature,DE_radiation_direct_horizontal,DE_radiation_diffuse_horizontal,hour,is_daylight
100,2014-01-05 04:00:00+00:00,2.920,0.0000,0.000,4,0
101,2014-01-05 05:00:00+00:00,2.954,0.0000,0.000,5,0
102,2014-01-05 06:00:00+00:00,2.965,0.0000,0.000,6,1
103,2014-01-05 07:00:00+00:00,2.983,0.0780,39.846,7,1
104,2014-01-05 08:00:00+00:00,3.355,0.8933,292.558,8,1


IMPORTANT FOR FUTURE! WHEN NORMALIZING, ONLY NORMALIZE IF IS_DAYLIGHT IS 1, THIS WAY IT WON'T GET DISTRACTED BY TOO MANY ZEROS!

In [ ]:
# Dealing with missing values strategy (for the future! can do it here)

# option 1: drop rows with missing values

#electricity_cleaned = electricity_df.dropna()
# Easy, no imputation, but could drop rare time windows

# option 2: forward fill, using most revent previous value, using time-series logic

#electricity_filled = electricity_df.fillna(method = 'ffill')
# time-consistent, less usesful for daily cycles (solar)

# option 3: feature-wise interpolation or median fill

#electricity_median = electricity_df.fillna(electricity_df.median(numeric_only = True))
#electricity_interp = electricity_df.interpolate(method = 'time')

Preprocessing logic below: change to python and run if using: (but rethink with the hint below, before executing)

Reminder: the missing values in question are solar & wind. These might need to be handled separately, because one is daily cycles (solar).

## 4: Outlier & Distribution Analysis

In [28]:
# Summary Statistics

display(electricity_df.describe())
display(weather_df.describe())

display(electricity_df.quantile([0.01, 0.99], numeric_only=True))
display(weather_df.quantile([0.01, 0.99], numeric_only=True))

,DE_solar_capacity,DE_solar_profile,DE_wind_capacity,DE_wind_profile,DE_wind_offshore_capacity,DE_wind_offshore_profile,DE_wind_onshore_capacity,DE_wind_onshore_profile
count,175167.000000,174783.000000,175167.000000,174869.000000,175167.000000,174869.000000,175167.000000,174879.000000
mean,42378.981566,0.101903,39974.878687,0.288840,3261.512779,2.238958,36713.377965,0.264938
std,4306.184780,0.156266,7261.521015,0.389282,1358.838591,4.135196,5960.534776,0.398901
min,37248.000000,0.000000,27913.000000,0.003300,667.000000,0.000000,27246.000000,0.003000
25%,38810.000000,0.000000,33737.000000,0.114600,2219.000000,0.206300,31519.000000,0.097400
50%,40941.000000,0.001800,39808.000000,0.221000,3115.000000,0.538700,36693.000000,0.191800
75%,46092.000000,0.163100,47730.000000,0.389500,4486.000000,0.899900,43243.000000,0.351700
max,50508.000000,0.690400,50452.000000,10.862000,5742.000000,14.988000,44710.000000,11.023000


,DE_temperature,DE_radiation_direct_horizontal,DE_radiation_diffuse_horizontal,hour,is_daylight
count,52584.000000,41773.000000,38092.000000,52584.000000,52584.000000
mean,9.921879,93.630721,145.120681,11.500000,0.583333
std,7.986186,203.467231,275.290585,6.922252,0.493011
min,-12.686000,0.000000,0.000000,0.000000,0.000000
25%,3.434750,0.000000,0.000000,5.750000,0.000000
50%,9.434500,0.000000,0.000000,11.500000,1.000000
75%,15.987000,55.951000,124.956250,17.250000,1.000000
max,35.479000,999.986000,999.931000,23.000000,1.000000


,DE_solar_capacity,DE_solar_profile,DE_wind_capacity,DE_wind_profile,DE_wind_offshore_capacity,DE_wind_offshore_profile,DE_wind_onshore_capacity,DE_wind_onshore_profile
0.01,37280.0,0.0000,28016.0,0.018400,683.0,0.00720,27334.0,0.0149
0.99,50478.0,0.5767,50329.0,0.885332,5723.0,13.61332,44607.0,0.8739


,DE_temperature,DE_radiation_direct_horizontal,DE_radiation_diffuse_horizontal,hour,is_daylight
0.01,-5.55551,0.0000,0.00000,0.0,0.0
0.99,28.04651,904.0766,970.33119,23.0,1.0


No outliers in solar profile or capacity (itghtly clustered)

Wind features have very suspicious max values -- might want to flag or clip this, but remember that when doing so, you could lose some predictive power?

#electricity_df['DE_wind_profile'] = electricity_df['DE_wind_profile'].clip(upper=0.885)

Weather data has long tails, skewed but natural (log transform)

In [29]:
electricity_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175167 entries, 0 to 175166
Data columns (total 16 columns):
 #   Column                                Non-Null Count   Dtype              
---  ------                                --------------   -----              
 0   utc_timestamp                         175167 non-null  datetime64[ns, UTC]
 1   cet_cest_timestamp                    175167 non-null  object             
 2   DE_load_actual_entsoe_transparency    175167 non-null  object             
 3   DE_load_forecast_entsoe_transparency  175071 non-null  object             
 4   DE_solar_capacity                     175167 non-null  int64              
 5   DE_solar_generation_actual            174783 non-null  object             
 6   DE_solar_profile                      174783 non-null  float64            
 7   DE_wind_capacity                      175167 non-null  int64              
 8   DE_wind_generation_actual             174869 non-null  object             
 9   DE_w

In [ ]:
# Visualize: for outlier detection

## 5: Merging

In [30]:
# Fix for electricity_df
if electricity_df.index.name != 'utc_timestamp':
    if 'utc_timestamp' in electricity_df.columns:
        electricity_df = electricity_df.set_index('utc_timestamp')
    else:
        raise ValueError("utc_timestamp not found in columns or index of electricity_df")

electricity_df = electricity_df.sort_index()

# Fix for weather_df
if weather_df.index.name != 'utc_timestamp':
    if 'utc_timestamp' in weather_df.columns:
        weather_df = weather_df.set_index('utc_timestamp')
    else:
        raise ValueError("utc_timestamp not found in columns or index of weather_df")

weather_df = weather_df.sort_index()


In [31]:
# Trim weather data to the time range of electricity data
start_time = electricity_df.index.min()
end_time = electricity_df.index.max()
weather_df = weather_df.loc[(weather_df.index >= start_time) & (weather_df.index <= end_time)]

# Reindex weather to match 15-min electricity timestamps
weather_aligned = weather_df.reindex(electricity_df.index)

# Merge both
merged_df = electricity_df.join(weather_aligned, how='left')
merged_df = merged_df.sort_index()

# Columns to fill
cols_to_fill = ['DE_temperature', 'DE_radiation_direct_horizontal', 'DE_radiation_diffuse_horizontal', 'hour']

# Option 1: Interpolated version
merged_interp = merged_df.copy()
merged_interp[cols_to_fill] = merged_interp[cols_to_fill].interpolate(method='time')

# Option 2: Forward-fill version
merged_ffill = merged_df.copy()
merged_ffill[cols_to_fill] = merged_ffill[cols_to_fill].fillna(method='ffill')

# Optional: Handle leading NaNs with bfill
merged_ffill[cols_to_fill] = merged_ffill[cols_to_fill].fillna(method='bfill')
merged_interp[cols_to_fill] = merged_interp[cols_to_fill].fillna(method='bfill')



/var/folders/yz/m7lkwlgn4p38457jgz6z55b40000gn/T/ipykernel_2076/4116762709.py:22: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_ffill[cols_to_fill] = merged_ffill[cols_to_fill].fillna(method='ffill')
/var/folders/yz/m7lkwlgn4p38457jgz6z55b40000gn/T/ipykernel_2076/4116762709.py:25: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_ffill[cols_to_fill] = merged_ffill[cols_to_fill].fillna(method='bfill')
/var/folders/yz/m7lkwlgn4p38457jgz6z55b40000gn/T/ipykernel_2076/4116762709.py:26: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_interp[cols_to_fill] = merged_interp[cols_to_fill].fillna(method='bfill')


In [32]:
merged_ffill.head()

,cet_cest_timestamp,DE_load_actual_entsoe_transparency,DE_load_forecast_entsoe_transparency,DE_solar_capacity,DE_solar_generation_actual,DE_solar_profile,DE_wind_capacity,DE_wind_generation_actual,DE_wind_profile,DE_wind_offshore_capacity,DE_wind_offshore_generation_actual,DE_wind_offshore_profile,DE_wind_onshore_capacity,DE_wind_onshore_generation_actual,DE_wind_onshore_profile,DE_temperature,DE_radiation_direct_horizontal,DE_radiation_diffuse_horizontal,hour,is_daylight
utc_timestamp,,,,,,,,,,,,,,,,,,,,
2015-01-01 07:15:00+00:00,2015-01-01T08:15:00+0100,40998.2,42497.11,37248,14.18,0.0004,27913,10433.26,0.3738,667,523.96,0.7855,27246,9909.3,0.3637,-1.046,88.773,519.464,8.0,NaN
2015-01-01 07:30:00+00:00,2015-01-01T08:30:00+0100,41120.9,42458.48,37248,49.02,0.0013,27913,10052.55,0.3601,667,525.52,0.7879,27246,9527.03,0.3497,-1.046,88.773,519.464,8.0,NaN
2015-01-01 07:45:00+00:00,2015-01-01T08:45:00+0100,41476.39,43085.68,37248,149.14,0.0040,27913,9962.65,0.3569,667,527.33,0.7906,27246,9435.32,0.3463,-1.046,88.773,519.464,8.0,NaN
2015-01-01 08:00:00+00:00,2015-01-01T09:00:00+0100,42120.4,44127.67,37248,340.85,0.0092,27913,9867.04,0.3535,667,527.28,0.7905,27246,9339.76,0.3428,-1.046,88.773,519.464,8.0,1.0
2015-01-01 08:15:00+00:00,2015-01-01T09:15:00+0100,42624.45,44906.5,37248,572.81,0.0154,27913,10067.22,0.3607,667,527.38,0.7907,27246,9539.84,0.3501,-1.046,88.773,519.464,8.0,NaN


In [33]:
# show rows 5-10 of merged_df
display(merged_ffill.iloc[0:10])

,cet_cest_timestamp,DE_load_actual_entsoe_transparency,DE_load_forecast_entsoe_transparency,DE_solar_capacity,DE_solar_generation_actual,DE_solar_profile,DE_wind_capacity,DE_wind_generation_actual,DE_wind_profile,DE_wind_offshore_capacity,DE_wind_offshore_generation_actual,DE_wind_offshore_profile,DE_wind_onshore_capacity,DE_wind_onshore_generation_actual,DE_wind_onshore_profile,DE_temperature,DE_radiation_direct_horizontal,DE_radiation_diffuse_horizontal,hour,is_daylight
utc_timestamp,,,,,,,,,,,,,,,,,,,,
2015-01-01 07:15:00+00:00,2015-01-01T08:15:00+0100,40998.2,42497.11,37248,14.18,0.0004,27913,10433.26,0.3738,667,523.96,0.7855,27246,9909.3,0.3637,-1.046,88.773,519.464,8.0,NaN
2015-01-01 07:30:00+00:00,2015-01-01T08:30:00+0100,41120.9,42458.48,37248,49.02,0.0013,27913,10052.55,0.3601,667,525.52,0.7879,27246,9527.03,0.3497,-1.046,88.773,519.464,8.0,NaN
2015-01-01 07:45:00+00:00,2015-01-01T08:45:00+0100,41476.39,43085.68,37248,149.14,0.0040,27913,9962.65,0.3569,667,527.33,0.7906,27246,9435.32,0.3463,-1.046,88.773,519.464,8.0,NaN
2015-01-01 08:00:00+00:00,2015-01-01T09:00:00+0100,42120.4,44127.67,37248,340.85,0.0092,27913,9867.04,0.3535,667,527.28,0.7905,27246,9339.76,0.3428,-1.046,88.773,519.464,8.0,1.0
2015-01-01 08:15:00+00:00,2015-01-01T09:15:00+0100,42624.45,44906.5,37248,572.81,0.0154,27913,10067.22,0.3607,667,527.38,0.7907,27246,9539.84,0.3501,-1.046,88.773,519.464,8.0,NaN
2015-01-01 08:30:00+00:00,2015-01-01T09:30:00+0100,43238.67,45320.58,37248,865.88,0.0232,27913,10133.99,0.3631,667,527.46,0.7908,27246,9606.53,0.3526,-1.046,88.773,519.464,8.0,NaN
2015-01-01 08:45:00+00:00,2015-01-01T09:45:00+0100,43869.42,45725.57,37248,1312.92,0.0352,27913,10047.66,0.3600,667,527.4,0.7907,27246,9520.26,0.3494,-1.046,88.773,519.464,8.0,NaN
2015-01-01 09:00:00+00:00,2015-01-01T10:00:00+0100,44285.28,46159.55,37248,1623,0.0436,27913,10208.11,0.3657,667,527.49,0.7908,27246,9680.62,0.3553,0.072,341.583,970.929,9.0,1.0
2015-01-01 09:15:00+00:00,2015-01-01T10:15:00+0100,44809.3,46744.9,37248,1890.12,0.0507,27913,10399.95,0.3726,667,525.01,0.7871,27246,9874.94,0.3624,0.072,341.583,970.929,9.0,NaN


In [ ]:
# For merged CSV (visualization of file)

#merged_ffill.to_csv("merged_dataset.csv", index=True)
#ran earlier, now showed up!

# Renamed after to merged_dataset_ffill (now found in processed folder)

In [34]:

# Save merged interpolated dataset to csv file in data/processed folder
merged_interp.to_csv("../data/processed/merged_dataset_interp.csv", index=True)


In [39]:
check = pd.read_csv("../data/processed/merged_dataset_ffill.csv", sep = ",", low_memory = False)
check.isnull().sum()

utc_timestamp                             0
cet_cest_timestamp                        0
DE_load_actual_entsoe_transparency        0
DE_load_forecast_entsoe_transparency     96
DE_solar_capacity                         0
DE_solar_generation_actual              384
DE_solar_profile                        384
DE_wind_capacity                          0
DE_wind_generation_actual               298
DE_wind_profile                         298
DE_wind_offshore_capacity                 0
DE_wind_offshore_generation_actual      298
DE_wind_offshore_profile                298
DE_wind_onshore_capacity                  0
DE_wind_onshore_generation_actual       288
DE_wind_onshore_profile                 288
DE_temperature                            0
DE_radiation_direct_horizontal            0
DE_radiation_diffuse_horizontal           0
hour                                      0
dtype: int64